# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau) **ET** un UID peut avoir plusieurs communes (inseecommune) 
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple




In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

### Les communes 

#### Name and info

In [2]:
%%sql --save int_edc__commune_udi
WITH
udi AS (
    SELECT
        inseecommune,
        cdreseau,
        de_partition,
        -- Prenons toujours le même nom de commune pour une inseecommune donnée
        MIN(nomcommune) AS nomcommune,
        -- Agréger les différentes valeurs de quartier en une liste sans doublons
        STRING_AGG(DISTINCT quartier, ', ') FILTER (WHERE quartier IS NOT NULL AND quartier != '') AS quartiers,
        -- Agréger les différentes valeurs de nomreseau en une liste sans doublons
        STRING_AGG(DISTINCT nomreseau, ', ') FILTER (WHERE nomreseau IS NOT NULL AND nomreseau != '') AS nomreseaux,
        -- Prendre la première date de début d'alimentation
        MIN(debutalim) AS debutalim
    FROM 
       edc_communes
    GROUP BY
        inseecommune,
        cdreseau,
        de_partition
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
)
    SELECT 
      udi.*,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.inseecommune = cog.commune_code_insee

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,cdreseau,de_partition,nomcommune,quartiers,nomreseaux,debutalim,code_departement,code_region
01001,001000556,2024,ABERGEMENT-CLEMENCIAT (L'),-,BDS ST DIDIER/CHALARONNE,2010-09-07,01,84
01002,001000369,2022,ABERGEMENT-DE-VAREY (L'),-,L'ABERGEMENT-DE-VAREY,2010-09-07,01,84
01004,001000249,2024,AMBERIEU-EN-BUGEY,St Germain_Brédevent,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,01,84
01005,001000850,2022,AMBERIEUX-EN-DOMBES,"Est, Ouest",BDS CHATANIER,2021-08-01,01,84
01006,001000235,2021,AMBLEON,-,AMBLEON,2010-09-07,01,84
01007,001000003,2020,AMBRONAY,-,AMBRONAY,2010-09-07,01,84
01008,001000254,2022,AMBUTRIX,Ambutrix centre,AMBUTRIX MAIRIE,2010-09-07,01,84
01009,001000338,2024,ANDERT-ET-CONDON,-,ANDERT-ET-CONDON-PUGIEU,2010-09-07,01,84
01010,001000260,2021,ANGLEFORT,le bourg,ANGLEFORT BOURG,2010-09-07,01,84
01011,001000870,2022,APREMONT,-,HBA LAC DE SYLANS,2010-09-07,01,84


#### LIST_REF_UDI_YEAR Pour chaque catégorie et années, on veut la liste complète des commmunes

In [3]:
%%sql  
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    )

SELECT
 annee, categorie
FROM 
annees
CROSS JOIN
cat

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie
2020,phénol
2021,phénol
2022,phénol
2023,phénol
2024,phénol
2020,non classé
2021,non classé
2022,non classé
2023,non classé
2024,non classé


In [4]:
%%sql --save LIST_REF_UDI_YEAR
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
),

udi AS (
    SELECT
      de_partition AS year,
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
    GROUP BY 
    1,2,3
)
    
SELECT DISTINCT
    annee, 
    categorie,
    commune_code_insee ,
 FROM
      udi
 FULL OUTER JOIN
       year_cat
 ON
    udi.year = year_cat.annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,commune_code_insee
2020,nitrite,01033
2020,nitrite,01258
2020,nitrite,04207
2020,nitrite,05040
2020,nitrite,05176
2020,nitrite,07240
2020,nitrite,08044
2020,nitrite,08101
2020,nitrite,09032
2020,nitrite,10115


In [5]:
%%sql --with LIST_REF_UDI_YEAR
SELECT
    *
FROM
   LIST_REF_UDI_YEAR
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,commune_code_insee
2023,cvm,07194
2021,cvm,07194
2020,cvm,07194
2024,cvm,07194
2022,cvm,07194


### Les résultats

#### mesures_cat

In [6]:
%%sql
WITH
resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    )
    
    SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,limitequal_float,unite,categorie
00200179905,TCEYTCL,0.0,<=10 µg/L,10.0,µg/L,hydrocarbure
00200179905,TCLC,0.0,None,None,None,non classé
00200179905,TCLEY,0.0,<=10 µg/L,10.0,µg/L,hydrocarbure
00200179905,TE,0.0,None,None,None,non classé
00200179905,TEAU,15.0,None,None,None,paramètre organoleptique
00200179905,TED,0.0,None,None,None,non classé
00200179905,TEMP_PH,15.0,None,None,None,paramètre organoleptique
00200179905,THM4,25.6,<=100 µg/L,100.0,µg/L,sous produit désinfection
00200179905,TI,0.0003,None,None,None,non classé
00200179905,TID,0.0003,None,None,None,non classé


#### mesures_cat_communes : on associe aux resultats la ref de prelevement et les uid associés

In [7]:
%%sql --save mesures_cat_communes
WITH 
/* mesures_cat*/
udi AS (
    SELECT
      de_partition AS year,
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
    GROUP BY 
    1,2,3
),
   
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mesures_cat AS (
    SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
)
/* END mesures_cat*/    
    
    SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat 
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi
    ON
    	udi.cdreseau = prelevement.cdreseau
        AND udi.year = extract( YEAR FROM prelevement.dateprel)

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,limitequal_float,unite,categorie,dateprel,commune_code_insee
00100119085,12DCLE,0.0,<=3 µg/L,3.0,µg/L,hydrocarbure,2020-02-14,01333
00100119085,ACTIK40,0.034,None,None,None,radioactivité,2020-02-14,01333
00100119085,ACTITR,0.0,None,None,None,radioactivité,2020-02-14,01333
00100119085,ADET,0.013,"<=0,1 µg/L",0.10000000149011612,µg/L,métabolite de pesticide,2020-02-14,01333
00100119085,ADET2,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,métabolite de pesticide,2020-02-14,01333
00100119085,ADETD,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,métabolite de pesticide,2020-02-14,01333
00100119085,ADSP,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,métabolite de pesticide,2020-02-14,01333
00100119085,ALTMICR,0.0,None,None,None,minéral,2020-02-14,01333
00100119085,AMTH,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides,2020-02-14,01333
00100119085,ATRZ,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides,2020-02-14,01333


In [8]:
%%sql --with mesures_cat_communes 
 SELECT 
   *
 FROM
   mesures_cat_communes
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,limitequal_float,unite,categorie,dateprel,commune_code_insee
00700174219,CLVYL,0.0,<=0.5 µg/L,0.5,µg/L,cvm,2021-03-31,07194
00700169098,CLVYL,0.0,<=0.5 µg/L,0.5,µg/L,cvm,2020-06-17,07194
00700167733,CLVYL,0.0,<=0.5 µg/L,0.5,µg/L,cvm,2020-04-15,07194
00700187059,CLVYL,3.6,<=0.5 µg/L,0.5,µg/L,cvm,2022-08-31,07194
00700187150,CLVYL,1.6,<=0.5 µg/L,0.5,µg/L,cvm,2022-09-14,07194
00700187169,CLVYL,0.37,<=0.5 µg/L,0.5,µg/L,cvm,2022-09-21,07194
00700187170,CLVYL,1.2,<=0.5 µg/L,0.5,µg/L,cvm,2022-09-21,07194
00700187171,CLVYL,3.1,<=0.5 µg/L,0.5,µg/L,cvm,2022-09-21,07194
00700187172,CLVYL,3.4,<=0.5 µg/L,0.5,µg/L,cvm,2022-09-21,07194
00700187629,CLVYL,0.0,<=0.5 µg/L,0.5,µg/L,cvm,2022-10-27,07194


#### mesures_cat_communes_year

In [9]:
%%sql --with mesures_cat_communes --save mesures_cat_communes_year

SELECT 
 extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
 categorie,
 commune_code_insee,
 SUM(1) AS nb_analyses,
 SUM(case
        when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
        else 0
    end ) AS nb_analyses_not_ok,
 SUM(case
        when limitequal_float is not NULL and valtraduite < limitequal_float then 1
        else 0
    end ) AS nb_analyses_ok,    
FROM
 mesures_cat_communes
GROUP BY 
    1,2,3

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,categorie,commune_code_insee,nb_analyses,nb_analyses_not_ok,nb_analyses_ok
2020,microbio,01158,192,64,0
2020,sous produit désinfection,01419,120,0,18
2020,microbio,01063,96,32,0
2020,minéral,01379,20,0,1
2020,sous produit désinfection,01386,26,0,6
2020,minéral,01430,69,0,3
2020,nitrite,01033,52,0,52
2020,microbio,01009,48,16,0
2020,minéral,01159,49,0,2
2020,minéral,01204,30,0,1


In [10]:
%%sql --with mesures_cat_communes_year 
 SELECT 
   *
 FROM
   mesures_cat_communes_year
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,commune_code_insee,nb_analyses,nb_analyses_not_ok,nb_analyses_ok
2020,cvm,07194,3,0,3
2021,cvm,07194,3,0,3
2022,cvm,07194,11,8,3
2023,cvm,07194,16,11,5
2024,cvm,07194,10,5,5


#### mesures_cat_communes_year_cvm

In [11]:
%%sql --with mesures_cat_communes_year 
SELECT
    annee,
    commune_code_insee,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'check case when'
    END AS resultat
FROM  
 mesures_cat_communes_year
WHERE
 categorie = 'cvm'

Running query in 'duckdb:///../../database/data.duckdb'

annee,commune_code_insee,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,83119,10,0,10,"<= 0,5 µg/L"
2020,83151,1,0,1,"<= 0,5 µg/L"
2020,83132,3,0,3,"<= 0,5 µg/L"
2020,83027,8,0,8,"<= 0,5 µg/L"
2020,83055,3,0,3,"<= 0,5 µg/L"
2020,83023,12,0,12,"<= 0,5 µg/L"
2020,83122,1,0,1,"<= 0,5 µg/L"
2020,83076,2,0,2,"<= 0,5 µg/L"
2020,84122,4,0,4,"<= 0,5 µg/L"
2020,84006,1,0,1,"<= 0,5 µg/L"


In [12]:
%%sql --with mesures_cat_communes_year 
SELECT
    annee,
    commune_code_insee,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'check case when'
    END AS resultat
FROM  
 mesures_cat_communes_year
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,commune_code_insee,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,07194,3,0,3,"<= 0,5 µg/L"
2021,07194,3,0,3,"<= 0,5 µg/L"
2022,07194,11,8,3,"> 0,5 µg/L"
2023,07194,16,11,5,"> 0,5 µg/L"
2024,07194,10,5,5,"> 0,5 µg/L"


**Pour un cas plus générique il faudra ajouter une condition sur categorie dans le CASE WHEN resultat**

#### mesures_cat_communes_year_cvm + Joint list annee_cat_communes list

In [13]:
%%sql --save mesures_cat_communes_year_resultat
WITH 
/* LIST_REF_UDI_YEAR  */
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
),

udi AS (
    SELECT
      de_partition AS year,
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
    GROUP BY 
    1,2,3
), 

LIST_REF_UDI_YEAR AS (    
SELECT DISTINCT
    annee, 
    categorie,
    commune_code_insee ,
 FROM
      udi
 FULL OUTER JOIN
       year_cat
 ON
    udi.year = year_cat.annee
),
/* END  LIST_REF_UDI_YEAR  */



    
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mesures_cat AS (
    SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
),

 mesures_cat_communes AS (
    SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat 
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi
    ON
    	udi.cdreseau = prelevement.cdreseau
    ),

 mesures_cat_communes_year AS(
    SELECT 
     extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
     categorie,
     commune_code_insee,
     SUM(1) AS nb_analyses,
     SUM(case
            when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
            else 0
        end ) AS nb_analyses_not_ok,
     SUM(case
            when limitequal_float is not NULL and valtraduite < limitequal_float then 1
            else 0
        end ) AS nb_analyses_ok,    
    FROM
     mesures_cat_communes
    GROUP BY 
        1,2,3
    )

SELECT
    LIST_REF_UDI_YEAR.annee,
    LIST_REF_UDI_YEAR.commune_code_insee,
    LIST_REF_UDI_YEAR.categorie,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'check case when'
    END AS resultat
FROM  
 LIST_REF_UDI_YEAR
LEFT JOIN 
 mesures_cat_communes_year
ON
    LIST_REF_UDI_YEAR.annee =  mesures_cat_communes_year.annee
    AND LIST_REF_UDI_YEAR.categorie =  mesures_cat_communes_year.categorie
    AND  LIST_REF_UDI_YEAR.commune_code_insee  =  mesures_cat_communes_year.commune_code_insee
WHERE
 LIST_REF_UDI_YEAR.categorie = 'cvm'

Your query is using one or more of the following snippets: LIST_REF_UDI_YEAR, mesures_cat_communes_year, mesures_cat_communes. JupySQL does not support snippet expansion within CTEs yet, CTE generation is disabled

Running query in 'duckdb:///../../database/data.duckdb'

annee,commune_code_insee,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,01004,cvm,55,0,55,"<= 0,5 µg/L"
2020,01058,cvm,10,0,10,"<= 0,5 µg/L"
2020,01022,cvm,10,0,10,"<= 0,5 µg/L"
2020,01332,cvm,20,0,20,"<= 0,5 µg/L"
2020,01431,cvm,10,0,10,"<= 0,5 µg/L"
2020,01155,cvm,15,0,15,"<= 0,5 µg/L"
2020,01284,cvm,40,0,40,"<= 0,5 µg/L"
2020,01175,cvm,40,0,40,"<= 0,5 µg/L"
2020,01094,cvm,40,0,40,"<= 0,5 µg/L"
2020,01375,cvm,50,0,50,"<= 0,5 µg/L"


#### mesures_cat_communes_year_cvm + Joint list annee_cat_communes list + int_edc__commune_udi 

In [14]:
%%sql --save mesures_cat_communes_year_cvm_with_cog
WITH 
/* int_edc__commune_udi */
udi AS (
    SELECT
        inseecommune AS commune_code_insee,
        cdreseau,
        de_partition AS year,
        -- Prenons toujours le même nom de commune pour une inseecommune donnée
        MIN(nomcommune) AS nomcommune,
        -- Agréger les différentes valeurs de quartier en une liste sans doublons
        STRING_AGG(DISTINCT quartier, ', ') FILTER (WHERE quartier IS NOT NULL AND quartier != '') AS quartiers,
        -- Agréger les différentes valeurs de nomreseau en une liste sans doublons
        STRING_AGG(DISTINCT nomreseau, ', ') FILTER (WHERE nomreseau IS NOT NULL AND nomreseau != '') AS nomreseaux,
        -- Prendre la première date de début d'alimentation
        MIN(debutalim) AS debutalim
    FROM 
       edc_communes
    GROUP BY
        inseecommune,
        cdreseau,
        de_partition
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),

int_edc__commune_udi AS (
    SELECT
      udi.*,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
),
/* END int_edc__commune_udi */
    
    
/* LIST_REF_UDI_YEAR  */
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
),

/*udi AS (
    SELECT
      de_partition AS year,
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
    GROUP BY 
    1,2,3
), */

LIST_REF_UDI_YEAR AS (    
SELECT DISTINCT
    annee, 
    categorie,
    commune_code_insee ,
 FROM
      udi
 FULL OUTER JOIN
       year_cat
 ON
    udi.year = year_cat.annee
),
/* END  LIST_REF_UDI_YEAR  */
   
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mesures_cat AS (
    SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
),

 mesures_cat_communes AS (
    SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat 
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi
    ON
    	udi.cdreseau = prelevement.cdreseau
    ),

 mesures_cat_communes_year AS(
    SELECT 
     extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
     categorie,
     commune_code_insee,
     SUM(1) AS nb_analyses,
     SUM(case
            when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
            else 0
        end ) AS nb_analyses_not_ok,
     SUM(case
            when limitequal_float is not NULL and valtraduite < limitequal_float then 1
            else 0
        end ) AS nb_analyses_ok,    
    FROM
     mesures_cat_communes
    GROUP BY 
        1,2,3
    )

SELECT
    LIST_REF_UDI_YEAR.annee,
    LIST_REF_UDI_YEAR.commune_code_insee,
    LIST_REF_UDI_YEAR.categorie,
    int_edc__commune_udi.nomcommune,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'check case when'
    END AS resultat
FROM  
 LIST_REF_UDI_YEAR
LEFT JOIN 
 mesures_cat_communes_year
ON
    LIST_REF_UDI_YEAR.annee =  mesures_cat_communes_year.annee
    AND LIST_REF_UDI_YEAR.categorie =  mesures_cat_communes_year.categorie
    AND  LIST_REF_UDI_YEAR.commune_code_insee  =  mesures_cat_communes_year.commune_code_insee
LEFT JOIN 
 int_edc__commune_udi
ON
    mesures_cat_communes_year.annee =  int_edc__commune_udi.year
    AND  mesures_cat_communes_year.commune_code_insee  =  int_edc__commune_udi.commune_code_insee    
WHERE
 LIST_REF_UDI_YEAR.categorie = 'cvm'

Your query is using one or more of the following snippets: LIST_REF_UDI_YEAR, mesures_cat_communes_year, int_edc__commune_udi, mesures_cat_communes. JupySQL does not support snippet expansion within CTEs yet, CTE generation is disabled

Running query in 'duckdb:///../../database/data.duckdb'

annee,commune_code_insee,categorie,nomcommune,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,56177,cvm,PLUVIGNER,25,0,25,"<= 0,5 µg/L"
2020,56030,cvm,CAMOEL,50,0,50,"<= 0,5 µg/L"
2020,56025,cvm,BRIGNAC,20,0,20,"<= 0,5 µg/L"
2020,56219,cvm,SAINT-GUYOMARD,25,0,25,"<= 0,5 µg/L"
2020,56145,cvm,NEANT-SUR-YVEL,30,0,30,"<= 0,5 µg/L"
2020,56076,cvm,GUERN,20,0,20,"<= 0,5 µg/L"
2020,57658,cvm,SOUCHT,5,0,5,"<= 0,5 µg/L"
2020,57564,cvm,RECHICOURT-LE-CHATEAU,45,0,45,"<= 0,5 µg/L"
2020,56090,cvm,INZINZAC-LOCHRIST,40,0,40,"<= 0,5 µg/L"
2020,57452,cvm,MAXE (LA),25,0,25,"<= 0,5 µg/L"


# Check

In [15]:
%%sql
SELECT
    de_partition as year,
    COUNT(DISTINCT inseecommune)
FROM 
 edc_communes
GROUP BY 
    1 
ORDER BY 
   1

Running query in 'duckdb:///../../database/data.duckdb'

year,count(DISTINCT inseecommune)
2020,34788
2021,34833
2022,34874
2023,34852
2024,34809


In [16]:
%%sql --with mesures_cat_communes_year_resultat  
 SELECT 
    annee,
    COUNT(DISTINCT commune_code_insee)
 FROM
   mesures_cat_communes_year_resultat
GROUP BY 
    1

Running query in 'duckdb:///../../database/data.duckdb'

annee,count(DISTINCT commune_code_insee)
2022,34874
2024,34809
2021,34833
2023,34852
2020,34788


In [18]:
%%sql --with mesures_cat_communes_year 
 SELECT 
   *
 FROM
   mesures_cat_communes_year
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,commune_code_insee,nb_analyses,nb_analyses_not_ok,nb_analyses_ok
2020,cvm,07194,3,0,3
2021,cvm,07194,3,0,3
2022,cvm,07194,11,8,3
2023,cvm,07194,16,11,5
2024,cvm,07194,10,5,5


In [21]:
%%sql --with mesures_cat_communes_year_resultat 
 SELECT 
   *
 FROM
   mesures_cat_communes_year_resultat
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,commune_code_insee,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,07194,cvm,15,0,15,"<= 0,5 µg/L"
2021,07194,cvm,15,0,15,"<= 0,5 µg/L"
2022,07194,cvm,55,40,15,"> 0,5 µg/L"
2023,07194,cvm,80,55,25,"> 0,5 µg/L"
2024,07194,cvm,50,25,25,"> 0,5 µg/L"


In [20]:
%%sql --with mesures_cat_communes_year_cvm_with_cog 
 SELECT 
   *
 FROM
   mesures_cat_communes_year_cvm_with_cog
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,commune_code_insee,categorie,nomcommune,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,07194,cvm,ROCHESSAUVE,15,0,15,"<= 0,5 µg/L"
2020,07194,cvm,ROCHESSAUVE,15,0,15,"<= 0,5 µg/L"
2021,07194,cvm,ROCHESSAUVE,15,0,15,"<= 0,5 µg/L"
2021,07194,cvm,ROCHESSAUVE,15,0,15,"<= 0,5 µg/L"
2022,07194,cvm,ROCHESSAUVE,55,40,15,"> 0,5 µg/L"
2022,07194,cvm,ROCHESSAUVE,55,40,15,"> 0,5 µg/L"
2023,07194,cvm,ROCHESSAUVE,80,55,25,"> 0,5 µg/L"
2023,07194,cvm,ROCHESSAUVE,80,55,25,"> 0,5 µg/L"
2024,07194,cvm,ROCHESSAUVE,50,25,25,"> 0,5 µg/L"
2024,07194,cvm,ROCHESSAUVE,50,25,25,"> 0,5 µg/L"


**ici on a des doublon pour les années. On comprendre pourquoi**